In [1]:
import json
import os 
import argparse
import wget 
import random 
import numpy as np 
from task_prompts import generic_task_prompts


In [4]:
parser = argparse.ArgumentParser(
                    prog='Dataset sampling')
parser.add_argument('--datasets_dir', default='./datasets', help="dir to retrieval datasets files and other resources") 
parser.add_argument('--out_dir', default='./dataset_sampled_out', help="dir to sampled dataset files") 
parser.add_argument('--data_sep_tags', default='none', help='none or tag, if data should be surrounded by tags')  
parser.add_argument('--instruct_sep_tags', default='none', help='none or tag, if instructions should be surrounded by tags')  
parser.add_argument('--sep_prompt', default='config_files/sep_prompt.txt', help='none, or a path to a file that contains defense prompt to explain/encourage separation')  


args, _ = parser.parse_known_args()

os.makedirs(args.out_dir, exist_ok=True)
dataset_out_name = 'dataset_out_clean_v2.json'
dataset_out_name = os.path.join(args.out_dir, dataset_out_name)
datasets_files = {'SQuAD': {'train': {'name': 'train-v2.0.json', 'url': 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json'},
                            'dev': {'name': 'dev-v2.0.json', 'url': 'https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json'} },

                  'hotpot': {'train': {'name': 'hotpot_train_v1.1.json' , 'url': 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json'},
                             'dev': {'name': 'hotpot_dev_fullwiki_v1.json', 'url': 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json'}}
                 }

In [5]:
def load_sep_prompt():
    #load prompt used to instruction the model how to do separation 
    if args.sep_prompt == 'none': 
        sep_prompt = ''
    else:
        with open(os.path.join(args.sep_prompt),"r") as f:
            sep_prompt = f.read()
    return sep_prompt 
sep_prompt = load_sep_prompt()

In [6]:
def download_datasets(datasets_dir, dataset):
    #download the squad and hotpot datasets if they are not downloaded
    os.makedirs(os.path.join(datasets_dir,dataset), exist_ok=True)
    for subset in datasets_files[dataset]:  
        if not os.path.isfile(os.path.join(datasets_dir,dataset,datasets_files[dataset][subset]['name'])):
            wget.download(datasets_files[dataset][subset]['url'], os.path.join(datasets_dir,dataset,datasets_files[dataset][subset]['name']))
    return 


In [7]:

#load datasets in a unified format. 
#format list of items. each is {'context': <TEXT PARAGRAPH> , 'questions': [{'question':, 'answer'} ... ]}
#questions is a list. squad has n questions (usually) for each context. 
#hotpot is usually one question with many paragraphs. Currently, just concatenate the paragraphs. 
def process_dataset(dataset_name, dataset_file):
    
    new_elements = []
    if dataset_name == 'SQuAD':
        for elem in dataset_file['data']:
            for par in elem['paragraphs']:
                paragraph = par['context']
                if len(par['qas']) == 0: continue 
                q_and_a = []
                for q in par['qas']:
                    if len(q['answers']) ==0: continue 
                    q_and_a.append({'question': q['question'],'answer':q['answers'][0]['text']})
                if len(q_and_a) != 0: new_elements.append({'context': paragraph, 'questions': q_and_a})
    elif dataset_name == 'hotpot':
        for elem in dataset_file:
            all_contexts = ''
            for context in elem['context'][0:3]:
                all_contexts += ' '.join(context[1])
            if all_contexts == '': continue 
            new_elements.append({'context':all_contexts, 'questions': [{'question': elem['question'],'answer':elem['answer']}]})
    return new_elements


In [8]:
datasets = ['SQuAD', 'hotpot']
retrieval_datasets = {}
for dataset in datasets: 
    download_datasets(args.datasets_dir, dataset)    
    dev_retrieval_orig_dataset = json.load(open(os.path.join(args.datasets_dir,dataset,datasets_files[dataset]['dev']['name'])))
    train_retrieval_orig_dataset = json.load(open(os.path.join(args.datasets_dir,dataset,datasets_files[dataset]['train']['name'])))

    retrieval_datasets[dataset] = {'dev': process_dataset(dataset, dev_retrieval_orig_dataset), 'train': process_dataset(dataset, train_retrieval_orig_dataset) }


In [9]:
#sample a main task.
#QA would sample a question from the dataset that belongs to the paragraph.
#otherwise, sample a prompt from one of the generic tasks. 
def sample_main_task(example_idx, task_type, dataset, subset ):
    orig_answer = ''
    question = random.choice(retrieval_datasets[dataset][subset][example_idx]['questions'])
    orig_task_prompt = question['question']
    orig_answer = question['answer']
    if task_type == 'mix':
        add_task = random.choice(list(generic_task_prompts.keys()))
        new_task_prompt = random.choice(generic_task_prompts[add_task])
        orig_task_prompt += '. After you answer the previous question, please do the following task as well: '
        orig_task_prompt += new_task_prompt
    return orig_task_prompt, orig_answer


In [10]:
def format_final_prompt(final_text_paragraph, orig_task_prompt):
    #final prompt.
    #final_text_paragraph is the output of the embedding process
    #if no orig instructions, then return the paragraph text 
    #otherwise, the format is: Defense prompt (if any) + instruction tags (if any) + data tags (if any) + data 
    final_prompt = ''
    if sep_prompt:
        final_prompt = final_prompt + sep_prompt + ' '
        
    if args.instruct_sep_tags != 'none':
        final_prompt = final_prompt + ' <'+args.instruct_sep_tags+'> ' + orig_task_prompt +' </' + args.instruct_sep_tags+'> '
    else:
        final_prompt = final_prompt + orig_task_prompt + ' '
        
    if args.data_sep_tags != 'none':
        final_prompt = final_prompt + ' <'+args.data_sep_tags+'> ' + final_text_paragraph + ' </' + args.data_sep_tags+'> '
    else:
        final_prompt = final_prompt + final_text_paragraph + ' '

    return final_prompt 
    

In [11]:

from tqdm import tqdm 
tasks = ['qa', 'mix']
new_samples = []
subset = 'train'
datasets = ['hotpot']
for dataset in datasets:
    samples = np.random.permutation(len(retrieval_datasets[dataset][subset]))
    for task in tasks: 
        for i in range(0, len(samples)):
            example_idx = samples[i]
            example_text_paragraph = retrieval_datasets[dataset][subset][example_idx]['context']
            orig_task_prompt, orig_task_answer = sample_main_task(example_idx, task, dataset, subset )
            
            final_aggregated_prompt = format_final_prompt(example_text_paragraph, orig_task_prompt)

            dataset_item = {'text_data_src': dataset, 
                        'split': subset, 
                        'text_idx': int(example_idx), 
                        'orig_text': example_text_paragraph,
                        'primary_task_type': task, 
                        'secondary_task_type': '', 
                        'secondary_task_context': '',
                        'primary_task_prompt': orig_task_prompt,
                        'primary_task_answer': orig_task_answer,
                        'secondary_task_prompt': '',
                        'secondary_has_answer': False, 
                        'secondary_witness': '', 
                        'embed_loc': '', 
                        'embed_method': '',
                        'gpt_model': '',
                        'instruct_sep_tags': args.instruct_sep_tags,
                        'data_sep_tags': args.data_sep_tags,
                        'embedding_prompt': '', 
                        'sep_prompt': sep_prompt, 
                        'final_text_paragraph': example_text_paragraph,
                        'annotated_paragraph': '',
                        'final_aggregated_prompt': final_aggregated_prompt}
        
            new_samples.append(dataset_item)
random.shuffle(new_samples)
#to have the same size as poisoned test examples
new_samples = new_samples[0:31134]



In [12]:
new_samples[1]

{'text_data_src': 'hotpot',
 'split': 'train',
 'text_idx': 28781,
 'orig_text': 'The Star Wars Corporation, Inc. (SWC) was a company founded by George Lucas in 1973.  It was a subsidiary of his Lucasfilm production company, set up to control various legal and financial aspects of his 1977 movie "Star Wars", including copyright, and sequel and merchandising rights.  By 1980, the company had been discontinued and its business was absorbed into the various divisions of its parent company Lucasfilm Ltd.Lucasfilm Ltd.  LLC is an American film and television production company based in the Letterman Digital Arts Center in San Francisco, California.  The studio is best known for creating and producing the "Star Wars" and "Indiana Jones" franchises, as well as its leadership in developing special effects, sound and computer animation for film.  Lucasfilm was founded by filmmaker George Lucas in 1971 in San Rafael, California; most of the company\'s operations were moved to San Francisco in 20

In [ ]:
with open(dataset_out_name, 'w') as fout:
    json.dump(new_samples , fout)